## Python for useRs

I decided to write this notebook to annotate my own experience in translating R to Python, as my current job is done almost entirely in Python. 

I'm a huge fan of the `tidyverse` and it's philosofy of reducing the programmer's time first and then the CPU time. So, for many things that I can acomplish in R with just a few **readable** lines of code, I had to google how to do them in `pandas` and very often the solution is an obscure piece of *hack*.

I imagine that this can happen to other people, and that's why I decided to share it. So, here we go!

In [1]:
# Activate the interface between R and python and also load the rpy2 extension to use R in the
# notebook cells
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython

### df1 %>% anti_join(df2, by = 'column')

**Problem:** you have two tables, lets say `df1` and `df2` and you want to filter the records on `df1` that aren't in `df2`, using a set of variables in common.

 * **Solution in R:** you use `dplyr::anti_join` 

 * **Solution in python:** you use merge with `how='outer'`, `indicator=True` and `suffixes=('', '_2')`, and then filter those lines in the result where the column `_merge` is equal to `'left_only'`
 
**Credit:** This [answer](https://stackoverflow.com/a/44200880/3665593) in stackoverflow gave me the main idea to solve the problem using `pandas`.

#### In R

In [12]:
%%R
suppressMessages(suppressWarnings(library(dplyr)))

df1 = tribble(
    ~id, ~group, ~x, ~y,
      1,      1, 10, 20,
      2,      1, 15, 40,
      3,      2, 20, 50,
      4,      2, 25, 60,
      5,      3, 25, 60,
      6,      3, 25, 60
)

df2 = tribble(
    ~id, ~group, ~x, ~y,
      1,      1, 11, 22,
      1,      2, 11, 22,
      3,      2, 21, 54,
      3,      3, 21, 54
)

df1_minus_df2 <- df1 %>% anti_join(df2, by = 'id')
df1_minus_df2

# A tibble: 4 x 4
     id group     x     y
  <dbl> <dbl> <dbl> <dbl>
1    2.    1.   15.   40.
2    4.    2.   25.   60.
3    5.    3.   25.   60.
4    6.    3.   25.   60.


#### In python

In [3]:
import pandas as pd

df1 = pd.DataFrame([
    [1, 1, 10, 20],
    [2, 1, 15, 40],
    [3, 2, 20, 50],
    [4, 2, 25, 60],
    [5, 3, 25, 60],
    [6, 3, 25, 60],
], columns=['id', 'group', 'x', 'y'])

df2 = pd.DataFrame([
    [1, 1, 11, 22],
    [1, 2, 11, 22],
    [3, 2, 21, 54],
    [3, 3, 21, 54],
], columns=['id', 'group', 'x', 'y'])

df1_minus_df2 = pd.merge(df1, df2, on='id', how='outer', suffixes=('', '_'), indicator=True)
df1_minus_df2 = df1_minus_df2[df1_minus_df2['_merge'] == 'left_only']
df1_minus_df2 = df1_minus_df2.loc[:, df1.columns]

df1_minus_df2

,id,group,x,y
2,2,1,15,40
5,4,2,25,60
6,5,3,25,60
7,6,3,25,60


#### Explanation
Lets see the python code step by step:

**Step 1**

In [4]:
df1_minus_df2 = pd.merge(df1, df2, on='id', how='outer', suffixes=('', '_'), indicator=True)
df1_minus_df2

,id,group,x,y,group_,x_,y_,_merge
0,1,1,10,20,1.0,11.0,22.0,both
1,1,1,10,20,2.0,11.0,22.0,both
2,2,1,15,40,NaN,NaN,NaN,left_only
3,3,2,20,50,2.0,21.0,54.0,both
4,3,2,20,50,3.0,21.0,54.0,both
5,4,2,25,60,NaN,NaN,NaN,left_only
6,5,3,25,60,NaN,NaN,NaN,left_only
7,6,3,25,60,NaN,NaN,NaN,left_only


I'll explain from obviuos to black magic what does each of the parameters mean:

1. With `on='id'` we tell pandas that we'll use the `id` column to do the join
2. With `how='outer'` we get the rows that are on `df1` *OR* `df2`
3. With `suffixes=('', '_2')` we tell pandas that the variables that come from `df1` shoud use `''` as suffix (no suffix at all) and the variables on `df2` should use `'_'` as a suffix. This parameter is necessary only if there are columns in `df2` that have the same name of some column in `df1`, but I preffer to use it always, just to be sure.
4. With indicator=True we tell pandas that we want a column called `_merge` that will tell us if a row in the result comes from `'both'` tables, from `'left_only'` (only `df1`) or from `right_only` (only `df2`).

**Step 2**

In [5]:
df1_minus_df2 = df1_minus_df2[df1_minus_df2['_merge'] == 'left_only']
df1_minus_df2

,id,group,x,y,group_,x_,y_,_merge
2,2,1,15,40,NaN,NaN,NaN,left_only
5,4,2,25,60,NaN,NaN,NaN,left_only
6,5,3,25,60,NaN,NaN,NaN,left_only
7,6,3,25,60,NaN,NaN,NaN,left_only


This step should be pretty straight forward. We'll use the `_merge` column to filter `'left_only'` rows.

**Step 3**

In [6]:
df1_minus_df2 = df1_minus_df2.loc[:, df1.columns]
df1_minus_df2

,id,group,x,y
2,2,1,15,40
5,4,2,25,60
6,5,3,25,60
7,6,3,25,60


Finally, we'll select only the columns that came from `df1`

#### Summarizing all in a function

I also created a function for a `semi_join`, which is kinda the opposite of an `anti_join`.

In [7]:
def anti_join(df1, df2, by):
    df1_minus_df2 = pd.merge(df1, df2, on=by, how='outer', suffixes=('', '_'), indicator=True)
    df1_minus_df2 = df1_minus_df2[df1_minus_df2['_merge'] == 'left_only']
    df1_minus_df2 = df1_minus_df2.loc[:, df1.columns]

    return df1_minus_df2


def semi_join(df1, df2, by):
    df1_and_df2 = pd.merge(df1, df2, on=by, how='inner', suffixes=('', '_'))
    df1_and_df2 = df1_and_df2.loc[:, df1.columns]

    return df1_and_df2

In [8]:
anti_join(df1, df2, by=['id', 'group'])

,id,group,x,y
1,2,1,15.0,40.0
3,4,2,25.0,60.0
4,5,3,25.0,60.0
5,6,3,25.0,60.0


In [9]:
semi_join(df1, df2, by=['id', 'group'])

,id,group,x,y
0,1,1,10,20
1,3,2,20,50


### group_by(...) %>% mutate(...)

#TODO: Explanation here

In [10]:
%%R
library(dplyr)

iris_z_score <- iris %>% 
    group_by(Species) %>%
    mutate(Sepal.Length_z_score = (Sepal.Length - mean(Sepal.Length)) / sd(Sepal.Length))

head(iris_z_score)

# A tibble: 6 x 6
# Groups:   Species [1]
  Sepal.Length Sepal.Width Petal.Length Petal.Width Species Sepal.Length_z_sco…
         <dbl>       <dbl>        <dbl>       <dbl> <fct>                 <dbl>
1         5.10        3.50         1.40       0.200 setosa               0.267 
2         4.90        3.00         1.40       0.200 setosa              -0.301 
3         4.70        3.20         1.30       0.200 setosa              -0.868 
4         4.60        3.10         1.50       0.200 setosa              -1.15  
5         5.00        3.60         1.40       0.200 setosa              -0.0170
6         5.40        3.90         1.70       0.400 setosa               1.12  


In [11]:
iris = r['iris']

def z_score(df, var):
    return (df[var] - df[var].mean()) / df[var].std()

iris_z_score = iris.copy()
iris_z_score['Sepal.Length_z_score'] = (iris_z_score
                                        .groupby('Species', group_keys=False)
                                        .apply(lambda g: z_score(g, 'Sepal.Length')))
iris_z_score.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Sepal.Length_z_score
0,5.1,3.5,1.4,0.2,setosa,0.266674
1,4.9,3.0,1.4,0.2,setosa,-0.300718
2,4.7,3.2,1.3,0.2,setosa,-0.868111
3,4.6,3.1,1.5,0.2,setosa,-1.151807
4,5.0,3.6,1.4,0.2,setosa,-0.017022
